In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
df = pd.read_csv("02_Test_Data_Set/sales_pos.csv")
df.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user       550068 non-null  int64  
 1   prod       550068 non-null  object 
 2   gender     550068 non-null  object 
 3   age_group  550068 non-null  object 
 4   job        550068 non-null  int64  
 5   city       550068 non-null  object 
 6   marital    550068 non-null  int64  
 7   prod_cat1  550068 non-null  int64  
 8   prod_cat2  376430 non-null  float64
 9   prod_cat3  166821 non-null  float64
 10  purchase   550068 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 46.2+ MB


## Q1.

In [4]:
len(df["prod"].unique())

3631

In [9]:
df_q1 = df.groupby("prod")["purchase"].sum().reset_index()
df_q1 = df_q1.sort_values("purchase", ascending = False)
df_q1

,prod,purchase
249,P00025442,27995166
1016,P00110742,26722309
2443,P00255842,25168963
582,P00059442,24338343
1745,P00184942,24334887
...,...,...
125,P00012942,1717
3117,P00325342,1656
3389,P00353042,1545
2962,P00309042,726


In [19]:
df.loc[df["prod"] == "P00025442", :]["job"].value_counts().idxmax()

4

## Q2.

In [31]:
df_q2 = df[["user", "marital", "age_group", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2.head(2)

,user,marital,age_group,prod_cat1,prod_cat2,prod_cat3
0,1,0,0-17,3,NaN,NaN
1,1,0,0-17,1,6.0,14.0


In [32]:
df_q2["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [33]:
df_q2 = df_q2.loc[df_q2["age_group"] == '26-35', :]
df_q2.head(2)

,user,marital,age_group,prod_cat1,prod_cat2,prod_cat3
5,3,0,26-35,1,2.0,NaN
9,5,1,26-35,8,NaN,NaN


In [34]:
df_q2.isna().sum()

user              0
marital           0
age_group         0
prod_cat1         0
prod_cat2     69427
prod_cat3    152645
dtype: int64

In [35]:
df_q2 = df_q2.fillna(0)
df_q2.isna().sum()

user         0
marital      0
age_group    0
prod_cat1    0
prod_cat2    0
prod_cat3    0
dtype: int64

In [36]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2.head(3)

,user,marital,age_group,prod_cat1,prod_cat2,prod_cat3
5,3,0,26-35,1,2,0
9,5,1,26-35,8,0,0
10,5,1,26-35,5,11,0


In [37]:
df_q2["prod_cat"] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]
df_q2.head(3)

,user,marital,age_group,prod_cat1,prod_cat2,prod_cat3,prod_cat
5,3,0,26-35,1,2,0,1-2-0
9,5,1,26-35,8,0,0,8-0-0
10,5,1,26-35,5,11,0,5-11-0


In [29]:
# apply lambda 사용, x.str.cat : concat 같이 다 붙여주는 함수
ser = df_q2.loc[:, "prod_cat1":"prod_cat3"].apply(lambda x: x.str.cat(sep = "-"), axis = 1)
ser

5          1-2-0
9          8-0-0
10        5-11-0
11         8-0-0
12         8-0-0
           ...  
550058    20-0-0
550059    19-0-0
550061    20-0-0
550064    20-0-0
550065    20-0-0
Length: 219587, dtype: object

In [ ]:
df_q2.groupby("marital")[""]

In [39]:
df_q2_m0 = df_q2.loc[df_q2["marital"] == 0, :]
df_q2_m1 = df_q2.loc[df_q2["marital"] == 1, :]

In [42]:
m0 = df_q2_m0.groupby("user")["prod_cat"].nunique().mean()
m1 = df_q2_m1.groupby("user")["prod_cat"].nunique().mean()
round(abs(m0 - m1), 2)

0.13

## Q3.

In [87]:
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


In [88]:
len(df["prod"].unique())

3631

In [89]:
dum = df[["user", "gender", "age_group", "job", "city", "marital"]].drop_duplicates()
dum.head(2)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
4,2,M,55+,16,C,0


In [90]:
dum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5891 entries, 0 to 243533
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user       5891 non-null   int64 
 1   gender     5891 non-null   object
 2   age_group  5891 non-null   object
 3   job        5891 non-null   int64 
 4   city       5891 non-null   object
 5   marital    5891 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 322.2+ KB


In [91]:
prod_cnt = df.groupby("user")["prod"].nunique().reset_index()
prod_cnt.head(2)

,user,prod
0,1,35
1,2,77


In [92]:
prod_cnt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5891 entries, 0 to 5890
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   user    5891 non-null   int64
 1   prod    5891 non-null   int64
dtypes: int64(2)
memory usage: 92.2 KB


In [93]:
# 인덱스 문제가 생겨서 dum 도 reset_index 필요
# 기존 "user" 컬럼이 정렬이 안돼있어서 sorting도 필요
# dum = dum.reset_index(drop = True).sort_values("user")
# pd.concat([dum, prod_cnt], axis = 1)

In [97]:
# merge를 쓰면 알아서 "user" 기준으로 일치하는 인스턴스만 모아줌
df_q2 = pd.merge(left = dum, right = prod_cnt, left_on = "user", right_on = "user")
df_q2

,user,gender,age_group,job,city,marital,prod
0,1,F,0-17,10,A,0,35
1,2,M,55+,16,C,0,77
2,3,M,26-35,15,A,0,29
3,4,M,46-50,7,B,1,14
4,5,M,26-35,20,A,1,106
...,...,...,...,...,...,...,...
5886,4588,F,26-35,4,C,0,20
5887,4871,M,18-25,12,C,0,12
5888,4113,M,36-45,17,C,0,20
5889,5391,M,26-35,7,A,0,7


In [98]:
tot_p = df.groupby("user")["purchase"].sum().reset_index()
df_q2 = pd.merge(left = df_q2, right = tot_p, left_on = "user", right_on = "user")
df_q2

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472
2,3,M,26-35,15,A,0,29,341635
3,4,M,46-50,7,B,1,14,206468
4,5,M,26-35,20,A,1,106,821001
...,...,...,...,...,...,...,...,...
5886,4588,F,26-35,4,C,0,20,140990
5887,4871,M,18-25,12,C,0,12,108545
5888,4113,M,36-45,17,C,0,20,213550
5889,5391,M,26-35,7,A,0,7,60182


In [99]:
df_q2["gender"] = df_q2["gender"].replace({"M":1, "F":0})
df_q2.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0-17,10,A,0,35,334093
1,2,1,55+,16,C,0,77,810472


In [100]:
df_q2["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [103]:
age_dict = {'0-17' : 0, '55+' : 6, '26-35' : 2, '46-50' : 4, '51-55' : 5, '36-45' : 3, '18-25': 1}
df_q2["age_group"] = df_q2["age_group"].replace(age_dict)
df_q2.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0,10,A,0,35,334093
1,2,1,6,16,C,0,77,810472


In [105]:
df_q2 = pd.get_dummies(df_q2, columns=["job", "city"])
df_q2.head(2)

,user,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,1,0,0,0,35,334093,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,1,6,0,77,810472,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [106]:
df_nor = MinMaxScaler().fit_transform(df_q2.iloc[:, 1:])
model = KMeans(n_clusters=7, random_state=123).fit(df_nor)

In [108]:
round(silhouette_score(X = df_nor, labels = model.labels_), 2)

0.18

In [10]:
df_q1 = df.groupby("prod").sum().reset_index()
df_q1

,prod,user,job,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,P00000142,3549402,9220,435,3456,4608.0,5760.0,12837476
1,P00000242,1161644,2860,140,752,1504.0,3384.0,3967496
2,P00000342,674888,1953,97,1220,3416.0,0.0,1296475
3,P00000442,284575,630,38,460,736.0,0.0,441173
4,P00000542,492005,1088,46,745,0.0,0.0,807212
...,...,...,...,...,...,...,...,...
3626,P0099442,601497,1591,58,1200,1600.0,0.0,2870383
3627,P0099642,35661,131,5,104,0.0,0.0,83710
3628,P0099742,380712,943,45,378,504.0,630.0,991948
3629,P0099842,320660,842,44,510,1224.0,1428.0,737312


In [12]:
df_q1.sort_values("purchase", ascending=False)

,prod,user,job,marital,prod_cat1,prod_cat2,prod_cat3,purchase
249,P00025442,4931763,13355,652,1615,3230.0,14535.0,27995166
1016,P00110742,4895727,13185,648,1612,3224.0,12896.0,26722309
2443,P00255842,4479169,11396,535,22128,0.0,0.0,25168963
582,P00059442,4320119,11638,577,8436,11248.0,22496.0,24338343
1745,P00184942,4245642,11770,597,1440,11520.0,24480.0,24334887
...,...,...,...,...,...,...,...,...
125,P00012942,4486,8,0,5,8.0,0.0,1717
3117,P00325342,2318,7,1,11,0.0,0.0,1656
3389,P00353042,3476,0,1,11,0.0,0.0,1545
2962,P00309042,4579,4,1,12,0.0,0.0,726


In [13]:
13355

13355

## Q2.

In [26]:
sample = df.loc[df["user"] == 1, ]
sample

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200
2,1,P00087842,F,0-17,10,A,0,12,NaN,NaN,1422
3,1,P00085442,F,0-17,10,A,0,12,14.0,NaN,1057
39180,1,P00085942,F,0-17,10,A,0,2,4.0,8.0,12842
78144,1,P00102642,F,0-17,10,A,0,4,8.0,9.0,2763
78145,1,P00110842,F,0-17,10,A,0,1,2.0,5.0,11769
78146,1,P00004842,F,0-17,10,A,0,3,4.0,12.0,13645
116848,1,P00117942,F,0-17,10,A,0,5,15.0,NaN,8839
116849,1,P00258742,F,0-17,10,A,0,5,NaN,NaN,6910


In [30]:
sample = sample.replace({np.nan : 0})
sample["prod_cat"] = sample["prod_cat1"].astype("int").astype("str") + "-" \
+ sample["prod_cat2"].astype("int").astype("str") + "-" \
+ sample["prod_cat3"].astype("int").astype("str")
sample

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase,prod_cat
0,1,P00069042,F,0-17,10,A,0,3,0.0,0.0,8370,3-0-0
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200,1-6-14
2,1,P00087842,F,0-17,10,A,0,12,0.0,0.0,1422,12-0-0
3,1,P00085442,F,0-17,10,A,0,12,14.0,0.0,1057,12-14-0
39180,1,P00085942,F,0-17,10,A,0,2,4.0,8.0,12842,2-4-8
78144,1,P00102642,F,0-17,10,A,0,4,8.0,9.0,2763,4-8-9
78145,1,P00110842,F,0-17,10,A,0,1,2.0,5.0,11769,1-2-5
78146,1,P00004842,F,0-17,10,A,0,3,4.0,12.0,13645,3-4-12
116848,1,P00117942,F,0-17,10,A,0,5,15.0,0.0,8839,5-15-0
116849,1,P00258742,F,0-17,10,A,0,5,0.0,0.0,6910,5-0-0


In [43]:
sample["prod_cat"].value_counts()

8-0-0      7
3-4-12     5
3-4-0      3
12-0-0     2
3-4-5      2
6-8-16     1
1-2-9      1
16-0-0     1
8-17-0     1
14-0-0     1
4-8-0      1
1-8-17     1
3-0-0      1
1-6-14     1
5-0-0      1
5-15-0     1
1-2-5      1
4-8-9      1
2-4-8      1
12-14-0    1
20-0-0     1
Name: prod_cat, dtype: int64

In [18]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [35]:
df_q2 = df.loc[df["age_group"] == "26-35", :]
df_q2 = df_q2[["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3
5,3,0,1,2.0,NaN
9,5,1,8,NaN,NaN


In [36]:
df_q2 = df_q2.replace({np.nan : 0})
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3
5,3,0,1,2.0,0.0
9,5,1,8,0.0,0.0


In [37]:
df_q2["prod_cat"] = df_q2["prod_cat1"].astype("int").astype("str") + "-" \
+ df_q2["prod_cat2"].astype("int").astype("str") + "-" \
+ df_q2["prod_cat3"].astype("int").astype("str")
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
5,3,0,1,2.0,0.0,1-2-0
9,5,1,8,0.0,0.0,8-0-0


In [50]:
df_q2.loc[df_q2["user"] == 3, :].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 5 to 506607
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user       29 non-null     int64  
 1   marital    29 non-null     int64  
 2   prod_cat1  29 non-null     int64  
 3   prod_cat2  29 non-null     float64
 4   prod_cat3  29 non-null     float64
 5   prod_cat   29 non-null     object 
dtypes: float64(2), int64(3), object(1)
memory usage: 1.6+ KB


In [56]:
df_q2.groupby("user")["prod_cat"].value_counts()

user  prod_cat
3     5-0-0       5
      1-2-0       4
      1-2-11      2
      1-2-14      2
      1-2-5       2
                 ..
6040  8-13-0      1
      8-13-15     1
      8-14-17     1
      8-9-14      1
      9-15-0      1
Name: prod_cat, Length: 85639, dtype: int64

## Q3.

In [5]:
df["user"].nunique()

5891

In [8]:
df_user = df[["user", "gender", "age_group", "job", "city", "marital"]].drop_duplicates()
df_user = df_user.reset_index(drop=True)
df_user.head(5)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,2,M,55+,16,C,0
2,3,M,26-35,15,A,0
3,4,M,46-50,7,B,1
4,5,M,26-35,20,A,1


In [9]:
len(df_user) # user의 고유개수 일치하는 지 확인

5891

In [13]:
df_user_agg1 = df.groupby("user")["prod"].nunique().reset_index() # 구매 상품 종류
df_user_agg1.head(2)

,user,prod
0,1,35
1,2,77


In [14]:
df_user_agg2 = df.groupby("user")["purchase"].sum().reset_index() # 총 구매금액
df_user_agg2.head(2)

,user,purchase
0,1,334093
1,2,810472


In [15]:
# concat도 가능하지만 user 번호에 확실하게 일치시키기 위해서 merge 사용
df_join = df_user.merge(df_user_agg1, left_on = "user", right_on = "user")
df_join = df_join.merge(df_user_agg2, left_on = "user", right_on = "user")
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [19]:
df_join["gender"] = df_join["gender"].replace({"M" : 1, "F": 0})
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0-17,10,A,0,35,334093
1,2,1,55+,16,C,0,77,810472


In [20]:
age_dict = {'0-17' : 0,
            '55+' : 6,
            '26-35' : 2,
            '46-50' : 4,
            '51-55' : 5,
            '36-45' : 3,
            '18-25' : 1}
age_dict

{'0-17': 0,
 '55+': 6,
 '26-35': 2,
 '46-50': 4,
 '51-55': 5,
 '36-45': 3,
 '18-25': 1}

In [21]:
df_join["age_group"] = df_join["age_group"].replace(age_dict)
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0,10,A,0,35,334093
1,2,1,6,16,C,0,77,810472


In [22]:
df_join = pd.get_dummies(data = df_join, columns = ["job", "city"])
df_join.head(2)

,user,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,1,0,0,0,35,334093,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,1,6,0,77,810472,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [27]:
df_join = df_join.drop(columns="user", axis=1)
df_join.head(2)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,6,0,77,810472,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [28]:
len(df_join.columns)

29

In [29]:
df_nor = MinMaxScaler().fit_transform(df_join)

In [30]:
model = KMeans(n_clusters = 7, random_state = 123).fit(df_nor)

In [31]:
model.labels_

array([3, 1, 3, ..., 1, 3, 5], dtype=int32)

In [32]:
round(silhouette_score(df_nor, model.labels_, random_state = 123), 2)

0.18